In [1]:
from Bio import SeqIO
#make a fasta file with reference genome and other variant genomes, align them with mafft

In [2]:
fasta = list(SeqIO.parse('corona-seq.fasta', 'fasta'))

In [3]:
for entry in fasta:
    print(entry.id)

Wuhan-Hu-1
B.1.1.7|Alpha
B.1.351|Beta
P.1,
B.1.617.2|Delta
BA.1|Omicron


In [4]:
seqs = {}
for entry in fasta:
    seqs[entry.id] = entry

In [5]:
for entry in fasta:
    print(len(entry.seq))

29903
29694
29885
29859
29821
29684


In [27]:
!mafft --auto corona-seq.fasta > corona-seq-ali.fasta


##### atgcfreq = 0.999994
nseq =  6
distance =  ktuples
iterate =  0
cycle =  2
sparsepickup = 0
nguidetree = 2
nthread = 0
sueff_global = 0.100000
generating a scoring matrix for nucleotide (dist=200) ... done
done
done
scoremtx = -1
Gap Penalty = -1.53, +0.00, +0.00

tuplesize = 6, dorp = d


Making a distance matrix ..

There are 30 ambiguous characters.
    1 / 6
done.

Constructing a UPGMA tree ... 
    0 / 6
done.

Progressive alignment 1/2... 
STEP     5 / 5 f
done.

Making a distance matrix from msa.. 
    0 / 6
done.

Constructing a UPGMA tree ... 
    0 / 6
done.

Progressive alignment 2/2... 
STEP     5 / 5 f
done.

disttbfast (nuc) Version 7.245 alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax=0.0
0 thread(s)


Strategy:
 FFT-NS-2 (Fast but rough)
 Progressive method (guide trees were built 2 times.)

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'mafft --man' and the mafft page.

The default g

In [6]:
fasta = list(SeqIO.parse('corona-seq-ali.fasta', 'fasta'))

In [7]:
seqs = {}
for entry in fasta:
    seqs[entry.id] = entry

In [8]:
for entry in fasta:
    print(len(entry.seq))

29916
29916
29916
29916
29916
29916


In [9]:
def gapped_pos(seq, pos):
    non_gap = 0
    gaps = 0
    
    for nt in seq:
        if nt != '-':
            non_gap += 1
        else:
            gaps += 1
        if non_gap == pos:
            return pos + gaps

In [26]:
gapped_pos("aaa-aa-a", 6)

8

In [ ]:
#21563..25384 we have the region of the spike protein from the original , we need to right a function to get the 
#spike protein region from the gapped or the aligned sequences!

In [12]:
gapped_pos (seqs['Wuhan-Hu-1'].seq, 21563)

21563

In [13]:
gapped_pos (seqs['Wuhan-Hu-1'].seq, 25384)

25393

In [14]:
spikes = {}
for seq in fasta:
    spikes[seq.id] = seq.seq[21563-1:25393]

In [15]:
for spike in spikes:
    print(spikes[spike][0:10])
    
#start codon is there

atgtttgttt
atgtttgttt
atgtttgttt
atgtttgttt
atgtttgttt
atgtttgttt


In [16]:
def get_mutations(initial, variant):
    seqs = list(zip(initial, variant))
    for pos, nt in enumerate(seqs):
        if nt[0] != nt[1]:
            print(nt[0].upper() + str(pos) + nt[1].upper())

In [18]:
get_mutations(spikes['Wuhan-Hu-1'],spikes['B.1.1.7|Alpha']) #comparing reference wuhan gneome spike protein 
#to alpha genome spike protein

T202-
A203-
C204-
A205-
T206-
G207-
T428-
T429-
A430-
A1509T
C1717A
A1849G
C2050A
C2155T
T2952G
G3360C


In [21]:
with open ('spikes.fasta', 'w') as f:
    for spike in spikes:
        out = spikes[spike].replace('-', '').translate()
        f.write('>' + spike + '\n')
        f.write(str(out).upper()+ '\n')
        
#this write a fasta file of all spike proteins of different variants
    

In [22]:
!ls

corona-seq-ali.fasta			mpxv-ha-sequences.fasta
corona-seq.fasta			Music
Desktop					Pictures
Documents				Public
Downloads				sars-spike-protein-analysis.ipynb
google-chrome-stable_current_amd64.deb	snap
miniconda3				spikes.fasta
Miniconda3-latest-Linux-x86_64.sh	Templates
mpx-seq-analystis-notebook.ipynb	Videos
mpxv-ha-sequences-ali.fasta


In [23]:
#now align amino acid sequnces of spike proteins
!mafft --auto spikes.fasta > spikes-ali.fasta


##### atgcfreq = 0.302478
nseq =  6
distance =  local
iterate =  16
cycle =  1
nthread = 0
lastonce = 0
done.
scoremtx = 1
    4 / 6

##### writing hat3
pairlocalalign (aa) Version 7.245 alg=L, model=BLOSUM62, 2.00, -0.10, +0.10, noshift, amax=0.0
0 thread(s)
minimumweight = 0.000500
nthread = 0
blosum 62 / kimura 200
sueff_global = 0.100000
Loading 'hat3' ... 
done.
done.
scoremtx = 1
Gap Penalty = -1.53, +0.00, +0.00
Loading 'hat2' ... done.
Constructing a UPGMA tree ... 
    0 / 6
done.

Progressive alignment ... 
STEP     5 /5 c
done.
tbfast (aa) Version 7.245 alg=A, model=BLOSUM62, 1.53, -0.00, -0.00, noshift, amax=0.0
0 thread(s)
minimumweight = 0.000500
autosubalignment = 0.000000
nthread = 0
randomseed = 0
blosum 62 / kimura 200
poffset = 0
niter = 16
sueff_global = 0.100000
Loading 'hat3' ... done.
done.
scoremtx = 1

    0 / 6
Segment   1/  1    1-1277
STEP 002-004-1  identical.    identical.   
Converged.

done
dvtditr (aa) Version 7.245 alg=A, model=BLOSUM62, 1.53, -0.00, 

In [24]:
spikes_aa = list(SeqIO.parse('spikes-ali.fasta', format = 'fasta'))

In [28]:
seqs = {}
for entry in spikes_aa:
    seqs[entry.id]= entry.seq

In [26]:
spikes_aa

[SeqRecord(seq=Seq('MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDL...HYT'), id='Wuhan-Hu-1', name='Wuhan-Hu-1', description='Wuhan-Hu-1', dbxrefs=[]),
 SeqRecord(seq=Seq('MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDL...HYT'), id='B.1.1.7|Alpha', name='B.1.1.7|Alpha', description='B.1.1.7|Alpha', dbxrefs=[]),
 SeqRecord(seq=Seq('MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDL...HYT'), id='B.1.351|Beta', name='B.1.351|Beta', description='B.1.351|Beta', dbxrefs=[]),
 SeqRecord(seq=Seq('MFVFLVLLPLVSSQCVNFTNRTQLPSAYTNSFTRGVYYPDKVFRSSVLHSTQDL...HYT'), id='P.1,', name='P.1,', description='P.1,', dbxrefs=[]),
 SeqRecord(seq=Seq('MFVFLVLLPLVSSQCVNLRTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDL...HYT'), id='B.1.617.2|Delta', name='B.1.617.2|Delta', description='B.1.617.2|Delta', dbxrefs=[]),
 SeqRecord(seq=Seq('MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDL...HYT'), id='BA.1|Omicron', name='BA.1|Omicron', description='BA.1|Omicron', dbxrefs=[])]

In [29]:
seqs['Wuhan-Hu-1']

Seq('MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDL...HYT')

In [37]:
def get_aa_mutations(initial, variant):
    seqs = list(zip(initial, variant))
    for pos, aa in enumerate(seqs):
        if aa[0] != aa[1]:
            print(aa[0].upper() + str(pos) + aa[1].upper())

#find mutations in spike protein amino acid sequence

In [38]:
get_aa_mutations (seqs['Wuhan-Hu-1'], seqs ['B.1.1.7|Alpha'])

H68-
V69-
Y143-
N503Y
A572D
D616G
P683H
T718I
S984A
D1120H


In [39]:
get_aa_mutations (seqs['Wuhan-Hu-1'], seqs ['B.1.351|Beta'])

D79A
D217G
L243-
L244-
A245-
K419N
E486K
N503Y
D616G
A703V


In [40]:
get_aa_mutations (seqs['Wuhan-Hu-1'], seqs ['BA.1|Omicron'])

A66V
H68-
V69-
T94I
G141D
V142-
Y143-
Y144-
N210-
L211I
-214E
-215P
-216E
G341D
S373L
S375P
S377F
K419N
N442K
G448S
S479N
T480K
E486A
Q495R
G498S
Q500R
N503Y
Y507H
T549K
D616G
H657Y
N681K
P683H
N766K
D798Y
N858K
Q956H
N971K
L983F


In [45]:
#make a list of mutations - we need to modify previous function
def get_aa_mutations(initial, variant):
    out = []
    seqs = list(zip(initial, variant))
    for pos, aa in enumerate(seqs):
        if aa[0] != aa[1]:
            out.append(aa[0].upper() + str(pos) + aa[1].upper())
    return out

In [46]:
get_aa_mutations (seqs['Wuhan-Hu-1'], seqs ['B.1.1.7|Alpha'])

['H68-',
 'V69-',
 'Y143-',
 'N503Y',
 'A572D',
 'D616G',
 'P683H',
 'T718I',
 'S984A',
 'D1120H']

In [47]:
#How many mutations are there? 
len(get_aa_mutations (seqs['Wuhan-Hu-1'], seqs ['B.1.1.7|Alpha']))


10

In [48]:
len(get_aa_mutations (seqs['Wuhan-Hu-1'], seqs ['B.1.351|Beta']))

10

In [49]:
len(get_aa_mutations (seqs['Wuhan-Hu-1'], seqs ['BA.1|Omicron']))

39

In [54]:
for item in seqs:
    print(item + ' '+ str(len(get_aa_mutations (seqs['Wuhan-Hu-1'], seqs[item]))))

Wuhan-Hu-1 0
B.1.1.7|Alpha 10
B.1.351|Beta 10
P.1, 12
B.1.617.2|Delta 10
BA.1|Omicron 39


In [ ]:
#it seems omicron has the highest mutation rate against reference genome!!!